# Figure S4: Regionalization differences (map)

In [1]:
import os 
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import shapely.geometry

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')
path_pmet = "/home/rooda/OneDrive/Projects/PatagoniaMet/"
path_geo = "/home/rooda/Datasets/GIS/"

## Data

In [2]:
q_data = xr.open_dataset("results/zenodo/Q_historical.nc")
q_data = q_data.resample(date = "YS").sum().mean(dim = "date")
q_data = q_data.to_dataframe().reset_index()
q_data = q_data.pivot(index="basin_id", columns="model", values="Q")

In [3]:
pmet_shape = gpd.read_file("data/GIS/Basins_Patagonia_all_data.gpkg") # Problem in Puelo 
pmet_shape = pmet_shape.set_index("gauge_id")
pmet_shape["total_area"] = pmet_shape.total_area*1e6 # in m2 
pmet_shape   = pd.concat([pmet_shape, q_data], axis = 1)
pmet_shape["Model_STD"]   = pmet_shape[["LSTM_OGGM_on", "LSTM_OGGM_off", "TUWmodel", "GR4J"]].std(axis = 1)
pmet_shape["Model_MEAN"]  = pmet_shape[["LSTM_OGGM_on", "LSTM_OGGM_off", "TUWmodel", "GR4J"]].mean(axis = 1)

## Plot elements

In [4]:
# basemap for background
geo_map = gpd.read_file(path_geo + "south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -54.99), (-76, -40.51), (-68.01, -40.51), (-68.01, -54.99), (-76, -54.99)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [5]:
pmet_shape_main = pmet_shape[pmet_shape["total_area"] > 5000*1e6].copy()
pmet_shape_main["basin_name"] = ["Santa Cruz                          ", "Puelo", "Yelcho", "Grey","Palena", "Pascua", "Aysen", "Baker", "Cisnes"]

In [6]:
from  plotly.colors import unlabel_rgb, hex_to_rgb
def binned_colorscale(seq, nr_swatches=5):
    
    if seq[0][0] == '#':
        arr_colors=np.array([hex_to_rgb(s) for s in seq])/255
    elif seq[0][0:3] == 'rgb':
        arr_colors = np.array([unlabel_rgb(s) for s in seq])/255 
    else:
        raise ValueError("a plotly colorscale is given either with hex colors or as rgb colors")
    n = len(seq)
    svals = [k/(n-1) for k in range(n)] #the scale values corresponding to the colors in seq
    grid = [k/(nr_swatches-1) for k in range(nr_swatches)]# define the scale values corresponding nr_swatches
    r, g, b = [np.interp(grid, svals, arr_colors[:, k]) for k in range(3)]  #np.interp interpolates linearly
    cmap_arr = np.clip(np.vstack((r, g, b)).T, 0, 1)
    new_colors = np.array(cmap_arr*255, dtype=int)
    discrete_colorscale = []
    N = len(new_colors+1)
    for k in range(N):
        discrete_colorscale.extend([[k/N, f'rgb{tuple(new_colors[k])}'], 
                                    [(k+1)/N,  f'rgb{tuple(new_colors[k])}']]) 
    return discrete_colorscale 

## Plot

In [7]:
titles = ["LSTM + OGGM on", "LSTM + OGGM off", "TUWmodel + OGGM", "GR4J + OGGM"]
fig = make_subplots(rows=1, cols=3, vertical_spacing = 0.01, horizontal_spacing = 0.01,
                    specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}]])

# colors
colorsc = binned_colorscale(["#fe7e0d", "#ffe9ba", "#1d78b4"], nr_swatches=15)

dtick = 2
x = list(range(-74, 0 + dtick, dtick))
y = list(range(-56, 0 + dtick, dtick))
xpos = -75.9
ypos = -55.9

## Basemap -----------------------------------------------------------------------------------------------
for x_plot in range(0,3):
        fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                                    colorscale = ["#EAEAF2", "#EAEAF2"],  showscale= False, marker_line_color ='white', marker_line_width=0.1), 
                                    row=1, col=x_plot+1)
    
        fig.add_trace(go.Scattergeo(lon = [lon + 0.25 for lon in x[1:-1]] + [xpos + 0.25] * (len(y) - 2),
                                lat = [ypos + 0.1] * (len(x) - 2) + [lat + 0.1 for lat in y[1:-1]],
                                showlegend = False,
                                text = x[1:-1] + y[1:-1], textfont=dict(size=10, color = "rgba(0,0,0,0.25)"),
                                mode = "text"), row=1, col=x_plot+1)

fig.add_scattergeo(geojson = eval(pmet_shape_main['geometry'].to_json()), locations = pmet_shape_main.index, text = pmet_shape_main['basin_name'], mode = 'text', 
                           showlegend = False, textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

# Plot
fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = (1-(pmet_shape['LSTM_OGGM_off']/pmet_shape['LSTM_OGGM_on']))*100, 
                            colorscale = colorsc, marker_line_color ='white', marker_line_width=0.2, 
                            zmin = -80, zmax = 80, colorbar=dict(len=0.97, x = 1.01, title='Relative<br>difference<br>(%)', thickness=20, tickwidth=1)), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = (1-(pmet_shape['TUWmodel']/pmet_shape['LSTM_OGGM_on']))*100, 
                            colorscale = colorsc, marker_line_color ='white', marker_line_width=0.2, showscale=False,
                            zmin = -80, zmax = 80), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = (1-(pmet_shape['GR4J']/pmet_shape['LSTM_OGGM_on']))*100, 
                            colorscale = colorsc, marker_line_color ='white', marker_line_width=0.2, showscale=False,
                            zmin = -80, zmax = 80), row=1, col=3)

## Annotations -----------------------------------------------------------------------------------------------
fig.add_annotation(text="a) Only LSTM",  x=0.005, y=0.02,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="b) TUWmodel + OGGM", x=0.42, y=0.02,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="c) GR4J + OGGM", x=0.80, y=0.02,  xref = "paper", yref = "paper", showarrow=False)

## Layout -------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)

fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", 
                lonaxis_range=[-76, -68], lataxis_range=[-56, -40.5], 
                bgcolor = "rgb(255,255,255)", 
                showland = False, showcoastlines = False, showlakes = False, 
                lataxis_showgrid=True, lonaxis_showgrid=True, 
                lonaxis_dtick = dtick, lataxis_dtick = dtick, 
                lonaxis_gridcolor = "rgba(0,0,0,0.1)", lataxis_gridcolor = "rgba(0,0,0,0.1)", 
                lonaxis_griddash = "dot", lataxis_griddash = "dot")

fig.update_layout(autosize = False, width = 950, height = 570, template = "seaborn", margin = dict(l=5, r=5, b=5, t=5))
fig.write_image("reports/figures/FigureS4_Regionalization_diff_map.png", scale = 4)
#fig.show()

## Text

In [ ]:
# comparison (LSTM OGGM on too low)
pmet_shape[["LSTM_OGGM_on", "LSTM_OGGM_off", "TUWmodel", "GR4J"]].mean() / pmet_shape["LSTM_OGGM_on"].mean()

In [ ]:
# comparison TODOOOOOOOOOOOOO
data = (1-(pmet_shape[["LSTM_OGGM_on", "LSTM_OGGM_off", "TUWmodel", "GR4J"]] / pmet_shape["LSTM_OGGM_on"].mean()))
"relative differences of {}".format((data.LSTM_OGGM_off * pmet_shape.total_area).sum()/pmet_shape.total_area.sum())